In [1]:
#!pip install pycaret



[notice] A new release of pip is available: 23.1.2 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
import logging
import os
import time
import pycaret
import json
from pycaret import classification
from pycaret.classification import *
from sklearn.feature_selection import mutual_info_classif, VarianceThreshold
from sklearn.model_selection import train_test_split

# Machine Learning Part

In [ ]:
FILENAME = "SAD"
DATA_PATH = "sad.csv"
TARGET = "ChanceofAdmit" 
KFOLD=10    
SELECTED_COLUMNS = ['Model', 'Accuracy', 'Prec.', 'Recall', 'F1', 'AUC']

# Load dataset
df = pd.read_csv(DATA_PATH)
df = df.drop(['Serial No.'], axis=1)
df.columns = df.columns.str.replace(' ', '')

# Transform class
df[TARGET] = df[TARGET].apply(lambda x: 1 if x >= 0.7 else 0)

# Separate features and target
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Feature Selection: Variance Filtering
selector_variance = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_variance = selector_variance.fit_transform(X)
variance_columns = X.columns[selector_variance.get_support()]

# Feature Selection: Mutual Information Filtering
mi_scores = mutual_info_classif(X_variance, y, discrete_features='auto')
mi_threshold = 0.01 
mi_mask = mi_scores > mi_threshold
X_mi = X_variance[:, mi_mask]
mi_columns = variance_columns[mi_mask]

# Create a DataFrame with the selected features
X_selected = pd.DataFrame(X_mi, columns=mi_columns)

# Create a new DataFrame with the selected features and selected class
df_selected = X_selected.copy()
df_selected[TARGET] = y

for gpu_available in [True, False]:
    for validation_type in ['kfold']:
        start_time = time.time()
           
        train_data = df_selected
        test_data = df_selected
        
        if(gpu_available == True):
            gpu = 1
        else:
            gpu = 0

        path = f"GPU_{gpu}_{FILENAME}_ML_VALIDATION_{validation_type}"
        # Create the dir if not exist
        os.makedirs(path, exist_ok=True)
        
        classification.setup(data = train_data, target = TARGET, use_gpu=gpu_available, fold=KFOLD)

        best_models = classification.compare_models(sort='Accuracy', cross_validation=True, exclude=['dummy'], n_select=15)

        end_time = time.time()
        execution_time_minutes = (end_time - start_time) / 60

        # Save model and hiperparams
        best_model = classification.compare_models(sort='Accuracy', cross_validation=True, exclude=['dummy'])
        best_model = pull()
        best_model = best_model[SELECTED_COLUMNS]           
        model_info = []

        for i,model in enumerate(best_models):
            model_name = f'{path}/{model.__class__.__name__}'
            save_model(model, model_name)
            
            # Get hiperparams
            model_params = model.get_params()
            
            # Add info
            model_info.append({
                'model_name': model_name,
                'hyperparameters': model_params
            })

        # Save
        with open(path+'/model_info.json', 'w') as file:
            json.dump(model_info, file, indent=4)

        # Save the DataFrame to a CSV file
        filename = f"GPU_{gpu}_{FILENAME}_ML_VALIDATION_{validation_type}_TIME_{execution_time_minutes:.2f}.csv"
        print(filename)
        best_model.round(4)
        best_model[best_model.select_dtypes(include=['number']).columns] *= 100
        best_model.to_csv(filename, index=False, sep=';', decimal=',', float_format='%.4f')


[codecarbon INFO @ 10:17:05] [setup] RAM Tracking...
[codecarbon INFO @ 10:17:05] [setup] GPU Tracking...
[codecarbon INFO @ 10:17:05] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:17:05] [setup] CPU Tracking...
[codecarbon WARNING @ 10:17:05] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 10:17:05] Energy consumed for RAM : 0.027081 kWh. RAM Power : 23.979151725769043 W
[codecarbon INFO @ 10:17:05] Energy consumed for all GPUs : 0.118308 kWh. Total GPU Power : 37.57463375179519 W
[codecarbon INFO @ 10:17:05] Energy consumed for all CPUs : 0.059313 kWh. Total CPU Power : 52.5 W
[codecarbon INFO @ 10:17:05] 0.204702 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:17:05] Energy consumed for RAM : 0.014392 kWh. RAM Power : 23.979151725769043 W
[codecarbon INFO @ 10:17:05] Energy consumed for all GPUs : 0.060355 kWh. Total GPU Power : 37.02972282466293 W
[codecarbon INFO @ 10:17:05] Energy consumed for all CPUs : 0.031517 kWh. To

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

,Description,Value
0,Session id,4109
1,Target,ChanceofAdmit
2,Target type,Binary
3,Original data shape,"(375, 8)"
4,Transformed data shape,"(375, 8)"
5,Transformed train set shape,"(281, 8)"
6,Transformed test set shape,"(94, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that mee

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8717,0.0000,0.8757,0.9105,0.8919,0.7337,0.7367,0.3120
lr,Logistic Regression,0.8681,0.9392,0.8820,0.9007,0.8901,0.7248,0.7278,0.3270
lda,Linear Discriminant Analysis,0.8681,0.9360,0.8699,0.9097,0.8885,0.7268,0.7299,0.3240
nb,Naive Bayes,0.8645,0.9392,0.8522,0.9202,0.8829,0.7224,0.7290,0.3130
rf,Random Forest Classifier,0.8432,0.9196,0.8640,0.8750,0.8684,0.6744,0.6772,0.5140
qda,Quadratic Discriminant Analysis,0.8400,0.9216,0.8404,0.8874,0.8613,0.6722,0.6772,0.3200
et,Extra Trees Classifier,0.8361,0.9112,0.8404,0.8839,0.8600,0.6621,0.6663,0.4770
catboost,CatBoost Classifier,0.8360,0.9143,0.8699,0.8607,0.8642,0.6567,0.6594,29.3320
knn,K Neighbors Classifier,0.8217,0.9047,0.8577,0.8526,0.8530,0.6257,0.6297,0.3440
lightgbm,Light Gradient Boosting Machine,0.8182,0.9122,0.8581,0.8437,0.8500,0.6190,0.6216,0.7160


...

,Description,Value
0,Session id,1504
1,Target,ChanceofAdmit
2,Target type,Binary
3,Original data shape,"(500, 8)"
4,Transformed data shape,"(500, 8)"
5,Transformed train set shape,"(350, 8)"
6,Transformed test set shape,"(150, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8829,0.9459,0.8952,0.9116,0.9020,0.7562,0.7601,0.2970
nb,Naive Bayes,0.8743,0.9418,0.8619,0.9250,0.8914,0.7425,0.7471,0.2950
lda,Linear Discriminant Analysis,0.8686,0.9432,0.8571,0.9221,0.8866,0.7304,0.7367,0.3150
ridge,Ridge Classifier,0.8629,0.0000,0.8524,0.9175,0.8819,0.7185,0.7251,0.2950
catboost,CatBoost Classifier,0.8629,0.9381,0.8810,0.8903,0.8849,0.7151,0.7168,0.5010
rf,Random Forest Classifier,0.8600,0.9252,0.8810,0.8882,0.8830,0.7084,0.7125,0.3220
et,Extra Trees Classifier,0.8543,0.9320,0.8714,0.8879,0.8778,0.6969,0.7015,0.3180
gbc,Gradient Boosting Classifier,0.8514,0.9173,0.8762,0.8783,0.8759,0.6905,0.6940,0.3360
qda,Quadratic Discriminant Analysis,0.8400,0.9357,0.8286,0.8987,0.8611,0.6730,0.6781,0.2960
lightgbm,Light Gradient Boosting Machine,0.8400,0.9241,0.8857,0.8554,0.8693,0.6630,0.6665,0.3530


Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8829,0.9459,0.8952,0.9116,0.9020,0.7562,0.7601,0.3090
nb,Naive Bayes,0.8743,0.9418,0.8619,0.9250,0.8914,0.7425,0.7471,0.3010
lda,Linear Discriminant Analysis,0.8686,0.9432,0.8571,0.9221,0.8866,0.7304,0.7367,0.3010
ridge,Ridge Classifier,0.8629,0.0000,0.8524,0.9175,0.8819,0.7185,0.7251,0.3020
catboost,CatBoost Classifier,0.8629,0.9381,0.8810,0.8903,0.8849,0.7151,0.7168,0.3080
rf,Random Forest Classifier,0.8600,0.9252,0.8810,0.8882,0.8830,0.7084,0.7125,0.3260
et,Extra Trees Classifier,0.8543,0.9320,0.8714,0.8879,0.8778,0.6969,0.7015,0.3170
gbc,Gradient Boosting Classifier,0.8514,0.9173,0.8762,0.8783,0.8759,0.6905,0.6940,0.3090
qda,Quadratic Discriminant Analysis,0.8400,0.9357,0.8286,0.8987,0.8611,0.6730,0.6781,0.3130
lightgbm,Light Gradient Boosting Machine,0.8400,0.9241,0.8857,0.8554,0.8693,0.6630,0.6665,0.3150


GPU_0_SAD_ML_VALIDATION_kfold_TIME_1.08_EMISSION_0.013719.csv
